# 07 - Generate statistics CSVs
Creating a separate notebook to calculate the statistics all in one pass so that I can complete a representative set of CSVs for each instrument class.

### Statistics for QARTOD tests in production

In [1]:
# Import libraries available from main conda channels or conda-forge
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os
import re
import warnings
warnings.filterwarnings("ignore")

# Import dask tools and ProgressBar
import dask
from dask.diagnostics import ProgressBar

# Import qartod_testing project functions
from qartod_testing.qc_flag_statistics import ooinet_gold_copy_request, \
    get_test_parameters, parse_qartod_executed, qartod_summary_expanded

# Import OOI library functions
from ooi_data_explorations.common import merge_frames
from ooinet.M2M import get_deployments, get_annotations

In [2]:
# Setup parameters needed to request data
refdes = "CP04OSSM-MFD35-06-PHSEND000"
method = "recovered_inst"
stream = "phsen_abcdef_instrument"

# Site, node, and sensor info from deconstructed reference designator
# [site, node, sensor] = refdes.split('-', 2)

In [3]:
# Routine in data_processing module from this project to download the gold copy THREDDs datasets
# Variable 'files' contains list of catalog URLs for downloaded datasets 
files = ooinet_gold_copy_request(refdes, method, stream)

In [4]:
# Load expected results data from external data folder
folder_path = os.path.join(os.path.abspath('../data/external'), method, stream, refdes)
expected_files = glob.glob(folder_path+'/*.nc')
expected_files.sort() # sorts local test files in alphanumeric order

In [5]:
expected_files
# get_deployments(refdes)

['/home/jovyan/code/qartod_testing/data/external/recovered_inst/phsen_abcdef_instrument/CP04OSSM-MFD35-06-PHSEND000/deployment0002_CP04OSSM-MFD35-06-PHSEND000-recovered_inst-phsen_abcdef_instrument_20150510T010459-20151013T060459.nc',
 '/home/jovyan/code/qartod_testing/data/external/recovered_inst/phsen_abcdef_instrument/CP04OSSM-MFD35-06-PHSEND000/deployment0004_CP04OSSM-MFD35-06-PHSEND000-recovered_inst-phsen_abcdef_instrument_20160527T220000-20161012T165959.nc',
 '/home/jovyan/code/qartod_testing/data/external/recovered_inst/phsen_abcdef_instrument/CP04OSSM-MFD35-06-PHSEND000/deployment0005_CP04OSSM-MFD35-06-PHSEND000-recovered_inst-phsen_abcdef_instrument_20161012T145959-20170309T055959.nc',
 '/home/jovyan/code/qartod_testing/data/external/recovered_inst/phsen_abcdef_instrument/CP04OSSM-MFD35-06-PHSEND000/deployment0006_CP04OSSM-MFD35-06-PHSEND000-recovered_inst-phsen_abcdef_instrument_20170607T015959-20171102T135959.nc',
 '/home/jovyan/code/qartod_testing/data/external/recovered_i

In [6]:
get_annotations(refdes)

,@class,id,subsite,node,sensor,method,stream,beginDT,endDT,annotation,exclusionFlag,source,qcFlag,parameters
0,.AnnotationRecord,937,CP04OSSM,MFD35,None,telemetered,None,1518199200000,1522327320000,Deployment 7: Communication with the MFN instr...,False,leila@marine.rutgers.edu,9,[]
1,.AnnotationRecord,1517,CP04OSSM,MFD35,None,telemetered,None,1525921218000,1540909800000,Instruments are presently disabled. Note: Inst...,False,leila@marine.rutgers.edu,0,[]
2,.AnnotationRecord,1628,CP04OSSM,MFD35,None,None,None,1557619200000,1569674280000,Deployment 10: Telemetered data from all MFN i...,False,cdobson@whoi.edu,0,[]
3,.AnnotationRecord,1557,CP04OSSM,None,None,telemetered,None,1548860400000,1554723540000,CP04OSSM D00009 buoy broke free and went adrif...,False,cdobson@whoi.edu,0,[]
4,.AnnotationRecord,355,CP04OSSM,MFD35,None,None,None,1476662400000,1491181218000,No telemetered data are expected because of gr...,False,leila@marine.rutgers.edu,0,[]
5,.AnnotationRecord,2408,CP04OSSM,MFD35,None,None,None,1605438000000,1617537960000,Deployment 12: The MFN began to experience vol...,False,cdobson@whoi.edu,9,[]
6,.AnnotationRecord,2869,CP04OSSM,MFD35,06-PHSEND000,None,None,1604503980000,1617537960000,Deploylment 12: Data points present seem reaso...,False,swhite@whoi.edu,3,[]
7,.AnnotationRecord,1362,CP04OSSM,None,None,telemetered,None,1445542680000,1463159880000,"Fuel cell impacted available instrument power,...",False,leila@marine.rutgers.edu,0,[]
8,.AnnotationRecord,3565,CP04OSSM,MFD35,06-PHSEND000,None,None,1445542680000,1463159880000,Deployment 3: This instrument was not deployed...,False,swhite@whoi.edu,0,[]
9,.AnnotationRecord,1821,CP04OSSM,MFD35,None,None,None,1573776000000,1604841180000,Deployment 11: MFN offline due to high voltage...,False,cdobson@whoi.edu,9,[]


#### Gross range test statistics

In [7]:
def collect_statistics(file_paths, test_name):
    """
    Calls other functions to calculate statistics from a set of files and a name of a QARTOD test. The statistics are organized in a DataFrame.
    
    Parameters:
    -----------
        file_paths: list of paths to each file that will have statistics calculated. File names must include "deployment00##".
        test_name: string of QARTOD test name, i.e. "gross_range", "climatology".
        
    Returns:
    --------
        statistics: Pandas DataFrame containing statistics on each parameter with a QARTOD test in order of deployment number, then statistics of the full record.
        
    Version 23 Aug 2023, Kylene M Cooley    
    """
    
    # Initialize empty dictionary for statistics
    statistics = {}
    
    # Create a copy of list of file paths for individual deployment statistics
    paths_copy = file_paths.copy()
    m = 0

    while len(paths_copy)>0:
        file = paths_copy[0]

        # get deployment from current file, then open local test and expected test datasets
        deployment = re.findall('deployment00[0-2][0-9]', file)[0][-2:]
        # here figure out how to get all files of a single deployment and do a merged dataset like below
        files_with_deployment_num = [x for x in paths_copy if f'deployment00{deployment}' in x]
        
        if len(files_with_deployment_num)>1:
            file_ds = [xr.open_dataset(single_file) for single_file in files_with_deployment_num]
            file_ds = merge_frames(file_ds)
        else:  
            file_ds = xr.open_dataset(file)

        # Get parameters that have QARTOD executed from expected test dataset
        test_parameters = get_test_parameters(file_ds)
        parameters = list(test_parameters.keys())

        # Separate QARTOD test flags in expected test dataset by QARTOD test name
        file_ds = parse_qartod_executed(file_ds, parameters)

        # Update summary statistics dictionary for each deployment, then for all deployments
        print("Evaluating statistics on QARTOD flags for deployment "f"{deployment}")
        summary_results = qartod_summary_expanded(file_ds, parameters, deployment, test_name)
        statistics.update({f"{m}" : summary_results })
        
        for x in files_with_deployment_num:
            paths_copy.remove(x)
        m += 1

    # Add entry to summary statistics for full data record after last deployments
    # Open all expected data files and create merged full dataset
    merged_ds = [xr.open_dataset(single_file) for single_file in file_paths]
    merged_ds = merge_frames(merged_ds)
    deployment = "all"

    # Summary of flags from merged dataset
    print("Evaluating statistics on QARTOD flags for all deployments")
    merged_ds = parse_qartod_executed(merged_ds, parameters)
    summary_results = qartod_summary_expanded(merged_ds, parameters, deployment, test_name)
    statistics.update({ "all" : summary_results })

    # Create data frame from dictionary and check contents
    statistics = pd.DataFrame.from_dict(statistics, orient='index')
    statistics = statistics.set_index('deployment')
    return statistics

In [8]:
gross_range_stats = collect_statistics(expected_files, "gross_range")
gross_range_stats

Evaluating statistics on QARTOD flags for deployment 02
Evaluating statistics on QARTOD flags for deployment 04
Evaluating statistics on QARTOD flags for deployment 05
Evaluating statistics on QARTOD flags for deployment 06
Evaluating statistics on QARTOD flags for deployment 07
Evaluating statistics on QARTOD flags for deployment 08
Evaluating statistics on QARTOD flags for deployment 09
Evaluating statistics on QARTOD flags for deployment 10
Evaluating statistics on QARTOD flags for deployment 11
Evaluating statistics on QARTOD flags for deployment 12
Evaluating statistics on QARTOD flags for deployment 13
Evaluating statistics on QARTOD flags for deployment 14
Evaluating statistics on QARTOD flags for deployment 15
Evaluating statistics on QARTOD flags for all deployments


,ph_seawater total,seawater good,seawater good %,seawater suspect,seawater suspect %,seawater fail,seawater fail %
deployment,,,,,,,
02,3750,2866,76.43,821,21.89,0,0.0
04,3308,2855,86.31,443,13.39,0,0.0
05,2872,2872,100.00,0,0.00,0,0.0
06,3565,2412,67.66,1126,31.58,0,0.0
07,3639,2188,60.13,1180,32.43,0,0.0
08,3719,3719,100.00,0,0.00,0,0.0
09,2324,2324,100.00,0,0.00,0,0.0
10,2935,2703,92.10,227,7.73,0,0.0
11,4231,4231,100.00,0,0.00,0,0.0


In [9]:
# Save data frames as CSVs
folder_path = os.path.join(os.path.abspath('../data/processed'), method, stream, refdes)
os.makedirs(folder_path, exist_ok=True)
gross_range_stats.to_csv(folder_path+f"/gross_range-{refdes}-flag_statistics.csv", na_rep='NaN', mode='w')

#### Climatology test statistics

In [10]:
climatology_stats = collect_statistics(expected_files, "climatology")
climatology_stats

Evaluating statistics on QARTOD flags for deployment 02
Evaluating statistics on QARTOD flags for deployment 04
Evaluating statistics on QARTOD flags for deployment 05
Evaluating statistics on QARTOD flags for deployment 06
Evaluating statistics on QARTOD flags for deployment 07
Evaluating statistics on QARTOD flags for deployment 08
Evaluating statistics on QARTOD flags for deployment 09
Evaluating statistics on QARTOD flags for deployment 10
Evaluating statistics on QARTOD flags for deployment 11
Evaluating statistics on QARTOD flags for deployment 12
Evaluating statistics on QARTOD flags for deployment 13
Evaluating statistics on QARTOD flags for deployment 14
Evaluating statistics on QARTOD flags for deployment 15
Evaluating statistics on QARTOD flags for all deployments


,ph_seawater total,seawater good,seawater good %,seawater suspect,seawater suspect %,seawater fail,seawater fail %
deployment,,,,,,,
02,3750,3117,83.12,631,16.83,0,0.0
04,3308,3028,91.54,279,8.43,0,0.0
05,2872,2871,99.97,1,0.03,0,0.0
06,3565,2530,70.97,1032,28.95,0,0.0
07,3639,1993,54.77,1646,45.23,0,0.0
08,3719,3706,99.65,13,0.35,0,0.0
09,2324,2324,100.00,0,0.00,0,0.0
10,2935,2640,89.95,295,10.05,0,0.0
11,4231,4208,99.46,23,0.54,0,0.0


In [11]:
# Save data frames as CSVs
folder_path = os.path.join(os.path.abspath('../data/processed'), method, stream, refdes)
os.makedirs(folder_path, exist_ok=True)
climatology_stats.to_csv(folder_path+f"/climatology-{refdes}-flag_statistics.csv", na_rep='NaN', mode='w')